# Big fimo output table cleanup

The "distribution_qvals_dmmpmm" notebook worked to create a big table containing information from five different motif databases. This notebook will clean up, collapse, and work with this big table. 

First, I'll import the table as a dataframe: 

In [2]:
import pandas as pd
import numpy as np
import seaborn as sb
import matplotlib.pyplot as plt
%matplotlib inline 

In [3]:
bigtable = pd.read_csv('../../output/concat.txt', sep='\t', index_col=0)

/Users/bergeric/miniconda3/lib/python3.5/site-packages/numpy/lib/arraysetops.py:395: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison
  mask |= (ar1 == a)


Collapsing over motif: 

In [4]:
bigtable['motif_hits'] = 0
grp=bigtable.groupby(['motif_FBgn','target_gene','source','#hits'])

In [5]:
agg=grp.agg({'min_pval': ['min'], 'motif_hits': 'count'})

In [6]:
agg.columns = ['min_pval', 'motif_hits'] 

In [7]:
agg2 = agg.reset_index()

In [8]:
agg2.columns = ['motif_FBgn','target_gene','source','pos_hits','min_pval', 'motif_hits']

In [9]:
agg2.head()

,motif_FBgn,target_gene,source,pos_hits,min_pval,motif_hits
0,FBgn0000014,FBgn0000003,flyReg,1,1,0.000027
1,FBgn0000014,FBgn0000003,onTheFly,1,2,0.000022
2,FBgn0000014,FBgn0000008,flyFactor,26,1,0.000118
3,FBgn0000014,FBgn0000008,flyReg,21,1,0.000027
4,FBgn0000014,FBgn0000008,idmmpmm,26,1,0.000027


Collapsing over database: 

In [10]:
#agg2['sum_pos_hits'] = 0
#agg2['sum_motif_hits'] = 0 
grp2 = agg2.groupby(['motif_FBgn','target_gene'])

In [11]:
agg3 = grp2.agg({'min_pval':['min'], 'pos_hits':['sum'],'motif_hits':['sum']})

In [12]:
agg3.columns = ['sum_pos_hits','min_pval', 'sum_motif_hits'] 

In [13]:
agg4 = agg3.reset_index()

In [14]:
#add column for motif_symbol
symbolmap = pd.read_table('/data/LCDB/lcdb-references/dmel/r6-11/gtf/dmel_r6-11.SYMBOL.csv', sep=',', na_values='NA', keep_default_na=False) 
newmap = symbolmap.drop_duplicates(subset='ENSEMBL', keep='first')
update = agg4.merge(newmap, left_on='motif_FBgn', right_on='ENSEMBL', how='left')
update = update.rename(columns={'SYMBOL': 'motif_symbol'})
trim = update[['motif_FBgn','motif_symbol','target_gene','min_pval','sum_motif_hits','sum_pos_hits']].copy()
trim.head()

,motif_FBgn,motif_symbol,target_gene,min_pval,sum_motif_hits,sum_pos_hits
0,FBgn0000014,abd-A,FBgn0000003,2,1,0.000048
1,FBgn0000014,abd-A,FBgn0000008,151,1,0.000337
2,FBgn0000014,abd-A,FBgn0000014,121,1,0.000337
3,FBgn0000014,abd-A,FBgn0000015,202,1,0.000337
4,FBgn0000014,abd-A,FBgn0000017,95,1,0.000337


In [15]:
#add column for gene_symbol
update2 = trim.merge(newmap, left_on='target_gene', right_on='ENSEMBL', how='left')
update2 = update2.rename(columns={'SYMBOL': 'gene_symbol'})
trim2 = update2[['motif_FBgn','motif_symbol','target_gene','gene_symbol','min_pval','sum_motif_hits','sum_pos_hits']].copy()
trim2.drop_duplicates(inplace=True)

In [16]:
trim2.groupby(['target_gene']).agg({'sum_motif_hits':['count']}).describe()

,sum_motif_hits
,count
count,17659.000000
mean,243.005776
std,43.787530
min,13.000000
25%,214.000000
50%,246.000000
75%,281.000000
max,299.000000


In [17]:
len(trim2.motif_FBgn.unique())

299

In [18]:
trim2.head()

,motif_FBgn,motif_symbol,target_gene,gene_symbol,min_pval,sum_motif_hits,sum_pos_hits
0,FBgn0000014,abd-A,FBgn0000003,NaN,2,1,0.000048
1,FBgn0000014,abd-A,FBgn0000008,a,151,1,0.000337
2,FBgn0000014,abd-A,FBgn0000014,abd-A,121,1,0.000337
3,FBgn0000014,abd-A,FBgn0000015,Abd-B,202,1,0.000337
4,FBgn0000014,abd-A,FBgn0000017,Abl,95,1,0.000337


In [24]:
#list of genes we have TF for/did RNAi on
TF_list = pd.read_table('../../data/list_of_tfs.txt', header=None)

In [25]:
TF_list.columns=['TF']

In [31]:
tfmerge = TF_list.merge(trim2, left_on='TF', right_on='motif_FBgn', how='inner')

In [41]:
len(newtrim.motif_FBgn.unique())

109

In [34]:
tfmerge.head()

,TF,motif_FBgn,motif_symbol,target_gene,gene_symbol,min_pval,sum_motif_hits,sum_pos_hits
0,FBgn0000054,FBgn0000054,Adf1,FBgn0000003,NaN,7,1,0.000138
1,FBgn0000054,FBgn0000054,Adf1,FBgn0000008,a,116,1,0.000105
2,FBgn0000054,FBgn0000054,Adf1,FBgn0000014,abd-A,75,1,0.000094
3,FBgn0000054,FBgn0000054,Adf1,FBgn0000015,Abd-B,160,1,0.000062
4,FBgn0000054,FBgn0000054,Adf1,FBgn0000017,Abl,91,1,0.000064


In [36]:
newtrim = tfmerge[['motif_FBgn','motif_symbol','target_gene','gene_symbol','min_pval','sum_motif_hits','sum_pos_hits']]

In [40]:
newtrim.to_csv('../../output/minpval_table')

In [37]:
matrix = newtrim[['motif_FBgn','target_gene','min_pval']]
matrix.set_index(['target_gene','motif_FBgn'], inplace=True)

In [38]:
final = matrix.unstack()

In [39]:
final.to_csv('../../output/matrix.txt', sep='\t')

In [86]:
trim2[(trim2['target_gene'] == 'FBgn0000276') & (trim2['motif_FBgn'] == 'FBgn0000014')]

,motif_FBgn,motif_symbol,target_gene,gene_symbol,min_pval,sum_motif_hits,sum_pos_hits
84,FBgn0000014,abd-A,FBgn0000276,CecA1,0.000102,5,6
